# **Face detection with OpenCV**
![대체 텍스트](https://pyimagesearch.com/wp-content/uploads/2018/02/deep_learning_face_detection_opencv.gif)



1.   OpenCV’s deep learning face detector is based on **the Single Shot Detector (SSD)** framework with a **ResNet** base network. 
2.   The network is defined and trained using the [Caffe Deep Learning framework](https://caffe.berkeleyvision.org/)





In [ ]:
#@title Import Libraries
# import the necessary packages
import os
import numpy as np
import cv2
import google.colab.patches

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
import six.moves.urllib.request 
import six 
import PIL

import base64, logging
import io
import IPython.display
import google.colab.output

Download the pre-trained face detection model, consisting of two files:
- The network definition (face_deploy.prototxt)
- The learned weights (res10_300x300_ssd_iter_140000.caffemodel) 
- https://dgschwend.github.io/netscope/quickstart.html






In [ ]:
#@title Load data and model
%%shell
mkdir faceDection; cd faceDection
mkdir model; cd model
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/face_deploy.prototxt 
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/face_deploy.caffemodel

cd ..; mkdir images; cd images
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/face_01.jpeg 

cd ..;mkdir videos; cd videos
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Videos/BTS.mp4
cd videos; mkdir save

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 28092  100 28092    0     0  64579      0 --:--:-- --:--:-- --:--:-- 3496k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 10.1M  100 10.1M    0     0  14.4M      0 --:--:-- --:--:-- --:--:-- 14.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  253k  100  253k    0     0   594k      0 --:--:-- --:--:-- --:--:-- 1020k
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("./faceDection/")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "model")
# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# Directory of video to run detection on
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")

In [ ]:
prototxtPath = os.path.sep.join([MODEL_DIR, "face_deploy.prototxt"])  
weightsPath = os.path.sep.join([MODEL_DIR, "face_deploy.caffemodel"]) 
faceNet = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)
# faceNet= cv2.dnn.readNet(weightsPath, prototxtPath,"caffe")

# **1**.Detection model demo by picture
>
https://deep-learning-study.tistory.com/299

In [ ]:
#@title Select an Input Image
SAMPLE_IMAGE = 'face_01.jpeg' #@param {type:"string"}
IMAGE_URL = ''  #@param {type:"string"}

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def download_and_resize_image(url,display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = six.moves.urllib.request.urlopen(url)
  image_data = response.read()
  image_data = six.BytesIO(image_data)
  pil_image = PIL.Image.open(image_data)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print(f"Image downloaded to {filename}.")
  if display:
    display_image(pil_image)
  return filename

if len(IMAGE_URL) !=0 :
  IMAGE_URL = download_and_resize_image(IMAGE_URL)
if len(SAMPLE_IMAGE) !=0 :
  SAMPLE_URL = os.path.join(IMAGE_DIR, SAMPLE_IMAGE)

image_url = IMAGE_URL or SAMPLE_URL

## Apply Detection model

In [ ]:
def detect_and_predict_face(frame, conf_threshold=0.25):
  height, width = frame.shape[:2]

  blob = cv2.dnn.blobFromImage(frame, 1.0,(300, 300), (104.0, 177.0, 123.0)) 
  faceNet.setInput(blob)
  detections = faceNet.forward()       

  for i in np.arange(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    
    if confidence > conf_threshold:
      box = detections[0, 0, i, 3:7]* np.array([width, height, width, height])   
      (startX, startY, endX, endY) = box.astype("int")
	
      text = f"{confidence * 100:.2f}%"
      y = startY - 10 if startY - 10 > 10 else startY + 10
      cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
      cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

In [ ]:
image = cv2.imread(image_url)
detect_and_predict_face(image)
google.colab.patches.cv2_imshow(image)

# **2**.Detection model demo by video

In [ ]:
#@title Videdo Capture
# Using a webcam to capture images for processing on the runtime.
# Source: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi

def data_uri_to_img(uri):
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = PIL.Image.open(io.BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8);
    return image
  except Exception as e:
    logging.exception(e)
    print('\n')
    return None

def video_to_data_url(filename):
    ext = filename.split('.')[-1]
    prefix = f"data:video/{ext};base64,"
    with open(filename, 'rb') as f:
        vidoe = f.read()
    return prefix + base64.b64encode(vidoe).decode()

# from base64 import b64decode
# playing webcam or video with javascript
def take_photo(filename='photo.jpg', quality=0.8):
  js = IPython.display.Javascript('''     
    async function takePhoto(filename, quality) {
                  
      const div = document.createElement('div');
      document.body.appendChild(div);

      const exit = document.createElement('button');
      exit.textContent = 'Exit';
      div.appendChild(exit);

      const video = document.createElement('video');
      video.style.display = 'block';
           
      if('photo.jpg' == filename){
        const stream = await navigator.mediaDevices.getUserMedia({video: true}); 
        video.srcObject = stream;   
      }else{
        video.src = filename;
        video.type="video/mp4"
      }
      await video.play();  
      div.appendChild(video);       
                   

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // when Exit button is clicked.   
      var isOpened = true; 
      var exitPromise = new Promise((resolve) => {exit.onclick = resolve});   
      exitPromise.then(()=>{isOpened = false; stream.getVideoTracks()[0].stop();});
      
      //when end of video
      var endPromise = new Promise((resolve) => {video.onended = resolve});   
      endPromise.then(()=>{isOpened = false; video.stop();});

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;

      for (let i = 0; isOpened; i++) {
        canvas.getContext("2d").clearRect(0, 0, canvas.width, canvas.height);
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // jsLog(i + "sending");
        // Call a python function and send this image
        google.colab.kernel.invokeFunction('notebook.run_faceDetection', [img], {});
        // jsLog(i + "SENT");

        // wait for X miliseconds second, before next capture
        await new Promise(resolve => setTimeout(resolve, 250));        
      }       
      div.remove();      
    }    
    ''')  
  # make the provided HTML, part of the cell
  IPython.display.display(js)
  #call the takePhoto() JavaScript function
  google.colab.output.eval_js(f"takePhoto('{filename}',{quality})")

frame_count = 0
writer = None
# InvokeFunction
# takes the numpy image and runs detection, then shows the results by visualizing
def run_faceDetection(uri): 
  global frame_count, writer

  image = data_uri_to_img(uri)     
  if writer is None:		
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')  
      writer = cv2.VideoWriter(outVideo, fourcc, 2, (image.shape[1], image.shape[0]), True)
  try:        
    detect_and_predict_face(image)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    frame_count+=1    
    name = f"{frame_count}.jpg"
    name = os.path.join(VIDEO_SAVE_DIR, name)
    cv2.imwrite(name, image)

    if writer is not None:
      writer.write(image)
  except Exception as e:
    logging.exception(e)
    print('\n')

# register this function, so JS code could call this
google.colab.output.register_callback('notebook.run_faceDetection', run_faceDetection)

## Apply Detection model

In [ ]:
%%shell
cd faceDection
rm ./videos/save/* 
rm ./videos/out.avi

In [ ]:
inVideo = os.path.join(VIDEO_DIR, "BTS.mp4")
outVideo= os.path.join(VIDEO_DIR, "out.avi")

In [ ]:
frame_count = 0
# data_url = video_to_data_url(inVideo)
data_url = 'photo.jpg'
try: 
  # put the JS code in cell and run it
  take_photo(data_url)  
  if writer is not None:
    writer.release()   
except Exception as e:
  logging.exception(e)
  print('\n')

writer = None

<IPython.core.display.Javascript object>

## Downlod  video

In [ ]:
from google.colab import files
files.download(outVideo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>